# Trabajo Practico Nº2 - Machine learning - Los convolucionales - Bellotti, Lopez, Trinchieri

### Imports iniciales

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications import VGG16

from sklearn.metrics import accuracy_score, confusion_matrix
%matplotlib inline

### Configuraciones iniciales de algunas constantes

In [ ]:
CATEGORIAS = 'buildings', 'forest', 'glacier', 'mountain', 'sea', 'street'

TRAIN_DIR = Path('train')
VALIDATION_DIR='validation'
TEST_DIR = Path('test')
SIZE = 150

### Análisis de la calidad de las imágenes que serán utilizadas para entrenar

Antes de comenzar a trabajar, revisamos en general las imagenes de train y nos encontramos con muchas imagenes que nada tenían que ver con la categoría a la que decian pertenecer, principalmente en **glacier**. Las mismas fueron removidas de dicho directorio y las guardamos en la carpeta deletes.

Un ejemplo para que se comprenda el tipo de imágenes que quitamos, es el de una persona disfrazada de dinosaurio en el gran cañón categorizada como **glacier**.

### Dividimos las imagenes de train en dos dataset de train y validation

Desarrollamos una función que extrae el primer 20% de las imagenes de cada categoría para que pasen a ser imagenes de validation. Utilizamos el porcentaje y no una cantidad fija para mantener las proporciones, además quitamos el primer 20% para que todos tengamos el mismo set de train y valdiation.

In [ ]:
import shutil

def dividir_train_validation_existente(carpeta_train, carpeta_validation, porcentaje_validacion=0.2):
    if not os.path.exists(carpeta_validation):
        os.makedirs(carpeta_validation)

    for categoria in os.listdir(carpeta_train):
        ruta_categoria_train = os.path.join(carpeta_train, categoria)
   
        if os.path.isdir(ruta_categoria_train):
            imagenes = [img for img in os.listdir(ruta_categoria_train) if os.path.isfile(os.path.join(ruta_categoria_train, img))]

            num_validacion = int(len(imagenes) * porcentaje_validacion)
    
            ruta_categoria_validation = os.path.join(carpeta_validation, categoria)
            if not os.path.exists(ruta_categoria_validation):
                os.makedirs(ruta_categoria_validation)

            for i in range(num_validacion):
                imagen = imagenes[i]
                ruta_imagen = os.path.join(ruta_categoria_train, imagen)
                shutil.move(ruta_imagen, os.path.join(ruta_categoria_validation, imagen))

    print("División completada: 20% de las imágenes movidas a validation.")

carpeta_train = 'train'
carpeta_validation = 'validation'

dividir_train_validation_existente(carpeta_train, carpeta_validation)

# Análisis exploratorio del conjunto de datos

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

def contar_imagenes_por_categoria(carpeta_train):
    conteo_categorias = {}

    for categoria in os.listdir(carpeta_train):
        ruta_categoria = os.path.join(carpeta_train, categoria)

        if os.path.isdir(ruta_categoria):
            cantidad_imagenes = len([img for img in os.listdir(ruta_categoria) if os.path.isfile(os.path.join(ruta_categoria, img))])
            conteo_categorias[categoria] = cantidad_imagenes

    return conteo_categorias

def graficar_distribucion(conteo_categorias):
    categorias = list(conteo_categorias.keys())
    cantidades = list(conteo_categorias.values())

    plt.figure(figsize=(10, 6))
    sns.barplot(x=categorias, y=cantidades, palette='viridis')


    plt.title('Distribución de Imágenes por Categoría en la Carpeta Test', fontsize=14)
    plt.xlabel('Categoría', fontsize=12)
    plt.ylabel('Cantidad de Imágenes', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.show()

conteo_categorias = contar_imagenes_por_categoria(TRAIN_DIR)

graficar_distribucion(conteo_categorias)

1. *Volumetría de los datos y distribución de las variables a predecir:*
   El conjunto de datos cuenta con un total aproximado de 14,000 imágenes, distribuidas en 6 categorías, de la siguiente manera:

   - *Buildings*: 15.6%
   - *Forest*: 16.2%
   - *Glacier*: 17.1%
   - *Mountain*: 17.9%
   - *Sea*: 16.2%
   - *Street*: 17.0%

   Las categorías están relativamente balanceadas, con diferencias menores en el número de imágenes entre clases. Ninguna categoría domina el conjunto de datos, lo que favorece el entrenamiento de un modelo equilibrado. Esto implica que no sería necesario realizar ajustes significativos para balancear las clases en este punto.

2. *Estructura y tipo de las imágenes:*
   - Las imágenes tienen un tamaño de *150x150 píxeles*.
   - El formato de las imágenes es JPG.
   - Las imágenes pertenecen a paisajes y escenas específicas de categorías bien diferenciadas, como edificios, naturaleza, y calles.


**Aclaración:** la distrución, luego de dividir train en dos sets nuevos, sigue siendo la misma proporción dado que nos quedamos con el 20% de cada categoría en validation y el 80% de cada categoría en train.

#### Al incio utilizamos un generador de imágenes reescaladas, con el brillo cambiado y rotadas

In [ ]:
images_reader_legacy = ImageDataGenerator(
     rescale=1/255,
     rotation_range=10,
     brightness_range=(0.5, 1.5)  
)

#### Dicho generador de imágenes no dió resultados óptimos, probamos con el siguiente, que tiene más características:

El mismo aplica una serie de transformaciones a las imágenes de entrada para realizar aumento de datos, mejorando la generalización del modelo. Normaliza los valores de los píxeles, rota, desplaza y aplica zoom a las imágenes de forma aleatoria, además de ajustar el brillo y permitir volteo horizontal. También realiza distorsiones de corte (shear) y rellena los píxeles vacíos generados por estas transformaciones con el valor más cercano.

In [ ]:
images_reader = ImageDataGenerator(
    rescale=1/255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=(0.5, 1.5),
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)
  
READ_PARAMS = dict(
    class_mode="categorical",
    classes=CATEGORIAS,
    target_size=(SIZE, SIZE),
    color_mode="rgb",
)

##### Inicializamos los generadores de train y validation para utilizar durante el entrenamiento de los modelos

In [ ]:
train = images_reader.flow_from_directory(TRAIN_DIR, **READ_PARAMS)
validation = images_reader.flow_from_directory(VALIDATION_DIR, **READ_PARAMS)

#### Ejemplos de imagenes generadas a utilizar

In [ ]:
def sample_images(dataset):
    plt.figure(figsize=(10, 10))
    images, labels = next(dataset)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(CATEGORIAS[np.argmax(labels[i])])
        plt.axis("off")

Ejemplos de train:

In [ ]:
sample_images(train)

Ejemplos de validation:

In [ ]:
sample_images(validation)

# Modelado

## Entrenamiento de los modelos

Definimos inicialmente la forma de las imagenes de entrada (input_shape) y el código encargado de almacenar los pesos del modelo al final de cada época, dado que son constantes que utilizaremos en los modelos.

In [ ]:
input_shape = (SIZE, SIZE, 3)

In [ ]:
model_weights_at_epochs = {}

class OurCustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        model_weights_at_epochs[epoch] = self.model.get_weights()

### **MLP** - Multi layer perceptron

#### Aclaración:

Este modelo lo vamos a entrenar solo para probar, de antemano sabemos que será un modelo que no dará buenos resultados con el tipo de problema que estamos trabajando... para ello necesitamos CNN (Redes convolucionales).

Definimos la **arquitectura** del modelo

In [ ]:
modelMLP = Sequential([
    Input(input_shape),
    
    Flatten(),

    Dense(500, activation='tanh'),
    Dropout(0.25),
    
    Dense(len(CATEGORIAS), activation='softmax'),
])

Lo compilamos

In [ ]:
modelMLP.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelMLP.summary()

Entrenamos el MLP con 5 epocas

In [ ]:
historyMLP = modelMLP.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

#### Conclusión:

Como comentamos anteriormente, los MLPs suelen ser menos eficientes que las redes convolucionales (CNN) para este tipo de tareas, ya que no están diseñados específicamente para capturar las jerarquías espaciales y las características locales en las imágenes. Y esta teoría se refuerza con la práctica, dado que si observamos la primer entrega que realizamos en el kaggle, el accuracy del modelo fue de 0.30 (referencia de la entrega de kaggle --> predicciones.csv)

### **Convolucional**

Modelo convolucional básico, utilizando funciones de activación relu y una capa densa con funcion de activación tanh (la cual no es indicada para clasificar 6 categorías, es preferible softmax o relu, como se utiliza en la siguiente capa densa).

In [ ]:
model_weights_at_epochs = {}

modelConvolucional = Sequential([
    Input(input_shape),

    Convolution2D(filters=10, kernel_size=(4, 4), strides=1, activation='relu'),
    Dropout(0.25),
    
    Convolution2D(filters=10, kernel_size=(4, 4), strides=1, activation='relu'),
    Dropout(0.5),
    
    MaxPooling2D(pool_size=(4, 4)),
    
    Flatten(),
    
    Dense(100, activation='tanh'),
    Dropout(0.25),
    
    Dense(len(CATEGORIAS), activation='softmax'),
])

In [ ]:
modelConvolucional.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelConvolucional.summary()

In [ ]:
model_weights_at_epochs = {}

historyConvolutional = modelConvolucional.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

(No generamos un .csv dado que los resultados no fueron buenos)

### Convolucional modificado

Este modelo convolucional utiliza más filtros (32, 64 y 128) para capturar características detalladas en diferentes niveles de abstracción, mientras que los kernels más pequeños de 3x3 permiten una mejor eficiencia en la detección de patrones locales. Las capas de MaxPooling con un tamaño de 2x2 reducen el tamaño de las características progresivamente, ayudando a prevenir el sobreajuste. La activación relu en las capas densas mejora la eficiencia en la propagación de gradientes, y el Dropout más alto (0.5) en las últimas capas evita el sobreajuste, lo que hace al modelo más robusto.

In [ ]:
modelConvolucionalV2 = Sequential([
    Input(shape=(150, 150, 3)),

    Convolution2D(filters=32, kernel_size=(3, 3), strides=1, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Convolution2D(filters=64, kernel_size=(3, 3), strides=1, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),

    Convolution2D(filters=128, kernel_size=(3, 3), strides=1, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    Dense(len(CATEGORIAS), activation='softmax'),
])

In [ ]:
modelConvolucionalV2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelConvolucionalV2.summary()

In [ ]:
model_weights_at_epochs = {}

historyConvolutionalV2 = modelConvolucionalV2.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

#### Conclusión acerca de los modelos convolucionales NO preentrenados

Hemos realizado diversas pruebas cambiando la cantidad de epocas de entrenamiento, modificando la arquitectura, los generadores de imagenes y demás, y los mejores resultados fueron de 0.77 (prediccionesVGG.csv) de accuracy en nuestras pruebas.
Consideramos que el uso de modelos preentrenados como VGG16 puede conducir a resultados significativamente mejores.

Por ello, damos paso a trabajar con los siguientes modelos:

### **VGG16**

Primero, probamos con un VGG16 básico con una arquitectura con función de activación tanh (que antes mencionamos que no servía), con el fin de validar si de todas maneras obteníamos mejores resultados que una red sin preentrenar y, para nuestra sorpresa, así fue.

In [ ]:
pretrained_model = VGG16(input_shape=input_shape, include_top=False)
pretrained_model.trainable = False

modelVGG16 = Sequential([
    pretrained_model,

    Flatten(),

    Dense(100, activation='tanh'),
    Dense(100, activation='tanh'),
    
    Dense(len(CATEGORIAS), activation='softmax'),
])

In [ ]:
modelVGG16.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelVGG16.summary()

In [ ]:
model_weights_at_epochs = {}

historyModelVGG16 = modelVGG16.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

Con VGG16 obtuvimos un accuracy del 0.80 (predicciones.csv) pero, aun así, queríamos mejorar y sabíamos que podríamos dado que la arquitectura definida era básica, por eso nos pusimos a investigar que arquitectura sería mejor y, a su vez, continuamos buscando modelos más potentes que nos permitan seguir mejorando.


El primer intento fue cambiar la función de activación por relu, que era mas adecuada al tipo de problema que estabamos trabajando. De esta manera obtuvimos un 0.86 (prediccionesVGGRelu.csv) de accuracy, lo que nos indicó que ibamos por buen camino.

In [ ]:
base_model = VGG16(input_shape=input_shape, include_top=False)
pretrained_model.trainable = False

modelVGG16_relu = Sequential([
    base_model,

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5), 
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='relu'),
    Dense(len(CATEGORIAS), activation='softmax')

])

In [ ]:
modelVGG16_relu.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelVGG16_relu.summary()

In [ ]:
model_weights_at_epochs = {}

historyModelVGG16_relu = modelVGG16_relu.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

Encontramos que leakyRelu como función de activación era mejor que relu sola dado que soluciona el problema "muerte de neuronas". 

Lo que ocurre es que las neuronas pueden quedar inactivas y no aprender si sus entradas son negativas, ya que la función devuelve cero para esas entradas. Leaky ReLU permite un pequeño valor negativo (0.01) para las entradas negativas, lo que permite que las neuronas continúen activas y aprendan.

In [ ]:
from tensorflow.keras.layers import LeakyReLU

pretrained_model = VGG16(input_shape=input_shape, include_top=False)
pretrained_model.trainable = False

modelVGG16_leaky_relu = Sequential([
    pretrained_model,
    Flatten(),
    Dense(128),
    LeakyReLU (alpha=0.1),
    Dense(len(CATEGORIAS), activation='softmax'),
])

modelVGG16_leaky_relu.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelVGG16_leaky_relu.summary()

model_weights_at_epochs = {}

historyLeakyRelu = modelVGG16_leaky_relu.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

Conclusiones VGG16:

A partir de la utilización de un modelo preentrenado ya pudimos observar una gran mejora en las respuestas del modelo, y modificando la función de activación principalmente es dónde encontramos la ventaja para el mismo, dado que cuando modifcamos la arquitectura (cantidad de capas, tipos, dropout, etc) no se notaba demasiado la mejora en accuracy.

El mejor resultado obtenido con el mismo es 0.87934 (prediccionesLeakyRelu2.csv).

De todas maneras, estabamos estancados porque no podíamos mejorar el accuracy. 

Por esto, investigamos y encontramos que existen más modelos preentrenados, y que son más potentes y eficientes para entrenar. Nosotros elegimos el siguiente:

### **Xception**

En principio, definimos el siguiente modelo base, al cual le fuimos modificando la arquitectura (cantidad y tipos de capas, droput o no, funciones de activación, etc) y modificando también los generadores de imágenes.

In [ ]:
from tensorflow.keras.applications import Xception

base_model = Xception(weights='imagenet', include_top=False, input_shape=(SIZE,SIZE,3))
base_model.trainable = False

modelXception = Sequential([
    base_model,

    Flatten(),

    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    
    Dense(len(CATEGORIAS), activation='softmax'),
])

modelXception.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelXception.summary()

model_weights_at_epochs = {}

historyXception= modelXception.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

Según lo comentado anteriormente, obtuvimos los siguientes resultados (por orden de entrega en Kaggle):

accuracy 0.889 - (prediccionesXception.csv)
accuracy 0.881 - (prediccionesXception6.csv)
accuracy 0.888 - (prediccionesXception11.csv)
accuracy 0.882 - (prediccionesXception19.csv) <-- Había overfiteado
accuracy 0.89 - (prediccionesXceptionEpoch8.csv)

El siguiente tenía le agregamos solo una capa densa más y mejoró: 
accuracy 0.906 - (prediccionesXceptionEpoch9.csv)

Luego, por ejemplo, limpiamos el dataset con lo comentado al comienzo y mejoró suitlmente:
accuracy 0.9088 - (prediccionesXceptionDataSetClean.csv)

**Pero, finalmente, conseguimos la siguiente arquitectura:**

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import LeakyReLU

base_model = Xception(weights='imagenet', include_top=False, input_shape=(SIZE,SIZE,3))
base_model.trainable = False

modelXceptionModified = Sequential([
    base_model,

    Flatten(),
    Dense(256),
    LeakyReLU(alpha=0.1),  
    Dropout(0.25),
    Dense(128),
    LeakyReLU(alpha=0.1),
    Dropout(0.25),
    Dense(100),
    LeakyReLU(alpha=0.1),
    Dense(len(CATEGORIAS), activation='softmax')
])

modelXceptionModified.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelXceptionModified.summary()

model_weights_at_epochs = {}

historyXceptionModified = modelXceptionModified.fit(
    train,
    epochs=10,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

La arquitectura tiene 4 capas densas que permiten aprender relaciones profundas, además utilizamos la función de activación LeakyReLU que, como comentamos antes, es mejor que relu porque asegura que las neuronas permanezcan activas durante el entrenamiento y no se "apaguen". 
Además, agregamos capas de Dropout para evitar el overfiteo, promoviendo que el modelo aprenda representaciones más robustas y generales. 
Y, finalmente, softmax como función de activación para la capa de salida.


Con esta arquitectura, el **accuracy del modelo es de 0.933 (prediccionesXceptionModificado.csv)**

### Función para análizar el entrenamiento y seleccionar la mejor época de entrenamiento

In [ ]:
model = historyXceptionModified


plt.plot(model.history['accuracy'], label='train')
plt.plot(model.history['val_accuracy'], label='validation')
plt.title('Accuracy over train epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.xlim(-1, 12)
plt.legend(loc='upper left')
plt.show()

#### Elegimos la que consideremos como la mejor epoca y nos quedamos con esos de pesos

In [ ]:
BEST_EPOCH = 4
# modelMLP.set_weights(model_weights_at_epochs[BEST_EPOCH])
# modelConvolucional.set_weights(model_weights_at_epochs[BEST_EPOCH])
# modelConvolucionalV2.set_weights(model_weights_at_epochs[BEST_EPOCH])
# modelVGG16.set_weights(model_weights_at_epochs[BEST_EPOCH])
# modelVGG16_relu.set_weights(model_weights_at_epochs[BEST_EPOCH])
# modelVGG16_leaky_relu.set_weights(model_weights_at_epochs[BEST_EPOCH])
# modelXception.set_weights(model_weights_at_epochs[BEST_EPOCH])

modelXceptionModified.set_weights(model_weights_at_epochs[BEST_EPOCH])


Ahora analizamos el error de ambos conjuntos para sacar nuestras propias conclusiones

In [ ]:
datasets = (
    ('train', images_reader.flow_from_directory(TRAIN_DIR, **READ_PARAMS, batch_size=-1)),
    ('validation', images_reader.flow_from_directory(VALIDATION_DIR, **READ_PARAMS, batch_size=-1)),
)

**Matriz de confusión**


Gráficamos esta matriz al finalizar cada entrenamiento de modelo con el fin de comprender en mayor profundidad sus respuestas.

(debemos modificar el model con el modelo entrenado)

In [ ]:
model = modelXceptionModified

for dataset_name, dataset in datasets:
    print('#' * 25, dataset_name, '#' * 25)

    batch_images, batch_labels = next(dataset)
    
    predictions = np.argmax(model.predict(batch_images), axis=-1)
    labels = np.argmax(batch_labels, axis=-1)
    
    print('Accuracy:', accuracy_score(labels, predictions))

    plt.figure(figsize=(3, 4))
        
    plt.xticks([0, 1, 2, 3, 4, 5], CATEGORIAS, rotation=45)
    plt.yticks([0, 1, 2, 3, 4, 5], CATEGORIAS)
    plt.xlabel('Predicted class')
    plt.ylabel('True class')

    plt.imshow(
        confusion_matrix(labels, predictions), 
        cmap=plt.cm.Blues,
        interpolation='nearest',
    )

    plt.show()

#### Pruebas con imágenes

Con la siguiente función probamos, a partir del modelo entrenado, que respuesta nos da a una imagen en particular.

In [ ]:
from IPython.display import Image, display

model = modelXceptionModified

def show_and_predict(image_path):
    image_array = img_to_array(load_img(image_path, target_size=(SIZE, SIZE)))
    inputs = np.array([image_array])
    predictions = model.predict(inputs)
    display(Image(image_path, width=500))
    print("Prediction:", CATEGORIAS[np.argmax(predictions)])
    print("Prediction detail:", predictions)
show_and_predict("./test/20070.jpg")

### Función para pasar a .csv las predicciones del modelo con el set de test

In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

CATEGORIA2 = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
SIZE = 150

def predict_images_from_directory(directory):
    results = [] 
    images = [] 

    for image_name in os.listdir(directory):
        image_path = os.path.join(directory, image_name)

        if image_name.endswith(".jpg"):
            image_array = img_to_array(load_img(image_path, target_size=(SIZE, SIZE)))
            images.append(image_array)

    inputs = np.array(images) / 255.0  

    predictions = modelXceptionModified.predict(inputs)

    for i, image_name in enumerate(os.listdir(directory)):
        if image_name.endswith(".jpg"):
            predicted_class = CATEGORIA2[np.argmax(predictions[i])]
            results.append([image_name, predicted_class])
    
    df = pd.DataFrame(results, columns=["ID", "Label"])
    df.to_csv("prediccionesXceptionModificaDos.csv", index=False)
    print("Predicciones guardadas en 'prediccionesXceptionModificaDos.csv'")

predict_images_from_directory('test')


# Conclusiones

Como era de esperar, dada la teoría, los modelos **MLP** tienen limitaciones para capturar las características espaciales y jerárquicas en imágenes. Esto los hace menos eficaces para problemas complejos de clasificación de imágenes en comparación con los modelos convolucionales, por ello obtuvimos tan solo un **0.3 de accuracy** y no nos eforzamos por conseguir algo mejor dado que nuestra hipótesis es que no ibamos a llegar a buen puerto. 

Por otro lado, en el caso de los modelos **convolucionales básicos**, si esperábamos que sean más efectivos para extraer características espaciales de las imágenes, gracias a las operaciones de convolución y agrupamiento. Sin embargo, las arquitecturas de las redes convolucionales simples que desarrollamos no fueron lo suficientemente buenas dado que el mejor accuracy conseguido fue de **0.769**.

Pero, por lo invesitgado, los modelos preentrenados como VGG16 y Xception han sido entrenados en grandes volúmenes de datos y son capaces de aprender representaciones más generales y sofisticadas, lo que les dió una ventaja con respecto a los convolucionales base que definimos nosotros. 

**VGG16** fue un modelo que entreno más lento y generó mejores resultados, pero no demasiados significativos, llegando a un accuracy máximo de **0.879**. 

**Xception**, según lo leído, es una arquitectura más avanzada basada en convoluciones separables, lo que le permite aprender representaciones más eficientes y generalizar mejor, logrando un rendimiento superior en comparación con VGG16. Dicha teoría se vió en la práctica dado que obtuvimos mejores resultados, como el accuracy del **0.93** y con un tiempo de entrenamiento menor.

En resumen, mientras que los MLP no sirvieron de mucho, los modelos convolucionales básicos tuvieron un mejor rendimiento del que esperabamos previamente. Y, por su parte, VGG16 fue menos eficiente y capaz de lo que creíamos, mientras que Xception nos sorprendió en su capacidad para aprender características ricas y generales, mejorando la precisión y la eficiencia del modelo.